In [1]:
# Import ML libraries
import pycaret
import xgboost

# Snowpark for Python
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import datediff, to_date, col, expr

# Import Misc
import json
import pandas as pd
# from pycaret.classification import setup, compare_models


In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open("connection.json"))
session = Session.builder.configs(connection_parameters).create()

In [3]:
# Connecting to the correct table
tableName = 'PURCHASE_ORDER_HISTORY'
dataframe = session.table(tableName)

# Calculation to find the lag between Planned Delivery from Actual Delivery
dataframe = dataframe.withColumn("target_feature",
                                    datediff('day', 
                                            col("DELIVERY_DATE_ML"), 
                                            col("FIRST_GR_POSTING_DATE_ML")))


# Example: Selecting specific columns
# This selects only a subset of columns. Adjust the column names as needed.
filtered_dataframe = dataframe.select(
    col("PURCHASE_DOCUMENT_ITEM_ID"), # ID for purchase order
    col("CREATE_DATE"),            # day purchase order was created
    col("COMPANY_CODE_ID"),           # copmany w/in INVISTA making purchase
    col("VENDOR_ID"),                 # ID of the vendor "we" are purchasing from
    col("POSTAL_CD"),                 # postal code associated w company code ID
    col("MATERIAL_ID"),               # ID of material being purchase
    col("SUB_COMMODITY_DESC"),        # description of sub commodity
    col("MRP_TYPE_ID"),               # determined if material is reordered manually or automatically
    col("PLANT_ID"),                  # ID of plant making purchase
    col("REQUESTED_DELIVERY_DATE"),# delivery date from requisition
    col("INBOUND_DELIVERY_ID"),       # ID for delivery
    col("INBOUND_DELIVERY_ITEM_ID"),  # ID of item w/in delivery
    col("PLANNED_DELIVERY_DAYS"),     # Amount of days expected to take
    col("FIRST_GR_POSTING_DATE"),  # expected delivery date        
    col("target_feature")             # Lag between Planned Delivery from Actual Delivery 
)


# Print a sample of the filtered dataframe to standard output.
filtered_dataframe.show()

# Optionally, you might want to filter rows based on some conditions
# Example: Filtering out rows where FIRST_GR_POSTING_DATE_ML is NULL
filtered_dataframe = filtered_dataframe.filter(col("FIRST_GR_POSTING_DATE").is_not_null())

# filtered_dataframe = filtered_dataframe[filtered_dataframe['PLANNED_DELIVERY_DAYS'] < 6]

# Show the DataFrame after filtering
filtered_dataframe.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PURCHASE_DOCUMENT_ITEM_ID"  |"CREATE_DATE"  |"COMPANY_CODE_ID"  |"VENDOR_ID"  |"POSTAL_CD"  |"MATERIAL_ID"  |"SUB_COMMODITY_DESC"                    |"MRP_TYPE_ID"  |"PLANT_ID"  |"REQUESTED_DELIVERY_DATE"  |"INBOUND_DELIVERY_ID"  |"INBOUND_DELIVERY_ITEM_ID"  |"PLANNED_DELIVERY_DAYS"  |"FIRST_GR_POSTING_DATE"  |"TARGET_FEATURE"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Assuming 'filtered_dataframe' is the DataFrame you've prepared in Snowflake
# Convert the Snowpark DataFrame to a Pandas DataFrame with consideration for NULL values

# Convert DataFrame to Pandas, handling NULL values by allowing float conversion
df = filtered_dataframe.fillna(0).to_pandas()  # This replaces NULL with 0 before conversion

Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "COMPANY_CODE_ID", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "VENDOR_ID", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "POSTAL_CD", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "SUB_COMMODITY_DESC", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "PLANNED_DELIVERY_DAYS", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>


In [5]:
df.head()

,PURCHASE_DOCUMENT_ITEM_ID,CREATE_DATE,COMPANY_CODE_ID,VENDOR_ID,POSTAL_CD,MATERIAL_ID,SUB_COMMODITY_DESC,MRP_TYPE_ID,PLANT_ID,REQUESTED_DELIVERY_DATE,INBOUND_DELIVERY_ID,INBOUND_DELIVERY_ITEM_ID,PLANNED_DELIVERY_DAYS,FIRST_GR_POSTING_DATE,TARGET_FEATURE
0,20,20210330,CA10,None,None,2100021412,Tolling,1,4007,20210331,185610163,20,0.0,20210330,-1
1,10,20210330,CN20,8010094262,201799,0,Telecommunications media services,0,1032,20210331,0,0,0.0,0,0
2,20,20210330,CN20,8010098163,200333,0,Maintenance Services,0,1032,20210406,0,0,0.0,20210511,35
3,10,20210330,CN16,8010019798,201604,0,Power Generation Equipment,0,1026,20210406,0,0,0.0,20210409,3
4,10,20210330,US10,8010099718,30165,2300006415,Custom Manufacturing,1,4016,20210409,185639199,10,5.0,20210413,4


In [6]:
df['TARGET_FEATURE']

0          -1
1           0
2          35
3           3
4           4
           ..
1139387    -2
1139388     0
1139389    -6
1139390    29
1139391    -1
Name: TARGET_FEATURE, Length: 1139392, dtype: int32

In [7]:
import numpy as np
import torch


In [8]:

# Handle missing values
# data.fillna(0, inplace=True)  

# Impute missing SUB_COMMODITY_DESC
df['SUB_COMMODITY_DESC'].fillna('Unknown', inplace=True)

# Convert categorical columns to one-hot encoding
df = pd.get_dummies(df, columns=['SUB_COMMODITY_DESC'])

# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder(sparse=False)  # Dense output
# encoded_data = encoder.fit_transform(df[['SUB_COMMODITY_DESC']])


In [9]:
import pandas as pd
import re

def clean_delivery_days(value):
    if isinstance(value, str):
        # Remove leading/trailing whitespace
        value = value.strip() 

        # Check for timestamp format and handle separately
        if re.match(r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}", value):  
            return ""  # Replace timestamps with NA or another placeholder
        else:
            return value
    else:
        return value

df['PLANNED_DELIVERY_DAYS'] = df['PLANNED_DELIVERY_DAYS'].apply(clean_delivery_days)


In [10]:
# Hash Alphanumeric columns
df['VENDOR_ID'] = df['VENDOR_ID'].apply(hash)
df['POSTAL_CD'] = df['POSTAL_CD'].apply(hash)
df['COMPANY_CODE_ID'] = df['COMPANY_CODE_ID'].apply(hash)




In [11]:
def remove_decimal(value):
    return value.split('.')[0]  # Split by the decimal and keep the integer part

df['PLANNED_DELIVERY_DAYS'] = df['PLANNED_DELIVERY_DAYS'].apply(remove_decimal)

for value in df['PLANNED_DELIVERY_DAYS'].unique():
    print(value, type(value), repr(value)) 



0 <class 'str'> '0'
5 <class 'str'> '5'
45 <class 'str'> '45'
14 <class 'str'> '14'
90 <class 'str'> '90'
1 <class 'str'> '1'
2 <class 'str'> '2'
10 <class 'str'> '10'
20 <class 'str'> '20'
100 <class 'str'> '100'
30 <class 'str'> '30'
3 <class 'str'> '3'
46 <class 'str'> '46'
6 <class 'str'> '6'
50 <class 'str'> '50'
4 <class 'str'> '4'
120 <class 'str'> '120'
60 <class 'str'> '60'
15 <class 'str'> '15'
31 <class 'str'> '31'
21 <class 'str'> '21'
7 <class 'str'> '7'
97 <class 'str'> '97'
19 <class 'str'> '19'
23 <class 'str'> '23'
13 <class 'str'> '13'
16 <class 'str'> '16'
17 <class 'str'> '17'
11 <class 'str'> '11'
59 <class 'str'> '59'
12 <class 'str'> '12'
34 <class 'str'> '34'
38 <class 'str'> '38'
25 <class 'str'> '25'
28 <class 'str'> '28'
9 <class 'str'> '9'
115 <class 'str'> '115'
87 <class 'str'> '87'
999 <class 'str'> '999'
18 <class 'str'> '18'
42 <class 'str'> '42'
24 <class 'str'> '24'
40 <class 'str'> '40'
65 <class 'str'> '65'
35 <class 'str'> '35'
8 <class 'str'> '8'


In [12]:
def convert_to_numeric(col):
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        print(f"Error converting column '{col}': Contains non-numeric values")

convert_to_numeric('PURCHASE_DOCUMENT_ITEM_ID')
convert_to_numeric('CREATE_DATE')
convert_to_numeric('VENDOR_ID')
convert_to_numeric('POSTAL_CD')
convert_to_numeric('MATERIAL_ID')
convert_to_numeric('MRP_TYPE_ID')
convert_to_numeric('PLANT_ID')
convert_to_numeric('INBOUND_DELIVERY_ID')
convert_to_numeric('INBOUND_DELIVERY_ITEM_ID')
convert_to_numeric('PLANNED_DELIVERY_DAYS')
convert_to_numeric('FIRST_GR_POSTING_DATE')


In [13]:
# Display the dataframe
df.head()


,PURCHASE_DOCUMENT_ITEM_ID,CREATE_DATE,COMPANY_CODE_ID,VENDOR_ID,POSTAL_CD,MATERIAL_ID,MRP_TYPE_ID,PLANT_ID,REQUESTED_DELIVERY_DATE,INBOUND_DELIVERY_ID,...,SUB_COMMODITY_DESC_Tools,SUB_COMMODITY_DESC_Transport operations services,"SUB_COMMODITY_DESC_Transportation, Storage, Mail Services",SUB_COMMODITY_DESC_Travel Services,SUB_COMMODITY_DESC_Tubes & Cores,SUB_COMMODITY_DESC_Unknown,SUB_COMMODITY_DESC_Valves,SUB_COMMODITY_DESC_Vehicles,SUB_COMMODITY_DESC_Waste Disposal Services,SUB_COMMODITY_DESC_Water Treatment Chemicals
0,20,20210330,-9134099118741362072,8794021623985,8794021623985,2100021412,1,4007,20210331,185610163,...,0,0,0,0,0,0,0,0,0,0
1,10,20210330,-5651926800411073115,-6786898889203162180,-6490539584081541328,0,0,1032,20210331,0,...,0,0,0,0,0,0,0,0,0,0
2,20,20210330,-5651926800411073115,6351911316643222019,3899556115142291491,0,0,1032,20210406,0,...,0,0,0,0,0,0,0,0,0,0
3,10,20210330,5345649740314450313,-5892833219993490273,-6223389847109689459,0,0,1026,20210406,0,...,0,0,0,0,0,0,0,0,0,0
4,10,20210330,-9066240345675022972,2189267853882615626,4393863124668179275,2300006415,1,4016,20210409,185639199,...,0,0,0,0,0,0,0,0,0,0


In [14]:

# Show types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139392 entries, 0 to 1139391
Data columns (total 87 columns):
 #   Column                                                     Non-Null Count    Dtype  
---  ------                                                     --------------    -----  
 0   PURCHASE_DOCUMENT_ITEM_ID                                  1139392 non-null  int16  
 1   CREATE_DATE                                                1139392 non-null  int32  
 2   COMPANY_CODE_ID                                            1139392 non-null  int64  
 3   VENDOR_ID                                                  1139392 non-null  int64  
 4   POSTAL_CD                                                  1139392 non-null  int64  
 5   MATERIAL_ID                                                1139392 non-null  int64  
 6   MRP_TYPE_ID                                                1139392 non-null  int8   
 7   PLANT_ID                                                   1139392 non-n

In [15]:
# Remove outliers
df = df[(df['TARGET_FEATURE'] < 100) & (df['TARGET_FEATURE'] > -100)]


In [16]:

# Create feature & target tensors on GPU
features = df.drop('TARGET_FEATURE', axis=1)
targets = df['TARGET_FEATURE']
X = torch.tensor(features.values.astype(np.float32))
y = torch.tensor(targets.values.astype(np.float32))

X

tensor([[ 2.0000e+01,  2.0210e+07, -9.1341e+18,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0000e+01,  2.0210e+07, -5.6519e+18,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.0000e+01,  2.0210e+07, -5.6519e+18,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.0000e+01,  2.0201e+07,  7.4181e+17,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0000e+01,  2.0201e+07,  1.3417e+18,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 3.0000e+01,  2.0201e+07, -9.1341e+18,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [17]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the data and transform it
X_normalized = scaler.fit_transform(X)

# Now X_normalized contains the normalized data


In [18]:
y

tensor([-1.,  0., 35.,  ..., -6., 29., -1.])

In [19]:

# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F



In [21]:
# define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X = X.to(device)
y = y.to(device)

# Add training and test data to the device
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)


In [22]:


class RegressionNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.hidden_size1 = 128  
        self.hidden_size2 = 64  

        self.linear1 = nn.Linear(input_size, self.hidden_size1)
        self.linear2 = nn.Linear(self.hidden_size1, self.hidden_size2)

        # Output layer for regression (no activation)
        self.output_layer = nn.Linear(self.hidden_size2, 1) 

    def forward(self, x):
        x = F.relu(self.linear1(x))  
        x = F.relu(self.linear2(x))
        x = self.output_layer(x)
        return x 


In [23]:
import torch
import torch.nn as nn

# Hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001
input_size = X.shape[1]  # Number of features


model = RegressionNetwork(input_size) 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Mean Absolute Error (MAE) loss
loss_fn = nn.L1Loss()
# loss_fn = nn.MSELoss()




In [24]:


# Data Preparation (If not using a DataLoader)
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# port to GPU
model.to(device)
loss_fn.to(device)





L1Loss()

In [25]:


# Train the model in batches
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        if torch.isnan(loss):
            print("NaN loss encountered. Exiting training loop.")
            break

        # loss.backward()
        
        # Gradient clipping
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/100], Step [100/13958], Loss: 3880996873175040.0000
Epoch [1/100], Step [200/13958], Loss: 2591036194947072.0000
Epoch [1/100], Step [300/13958], Loss: 4565754246594560.0000
Epoch [1/100], Step [400/13958], Loss: 1880706720464896.0000
Epoch [1/100], Step [500/13958], Loss: 1831160917262336.0000
Epoch [1/100], Step [600/13958], Loss: 1652394714726400.0000
Epoch [1/100], Step [700/13958], Loss: 1810927057895424.0000
Epoch [1/100], Step [800/13958], Loss: 1157363763511296.0000
Epoch [1/100], Step [900/13958], Loss: 2458042532626432.0000
Epoch [1/100], Step [1000/13958], Loss: 2640325776506880.0000
Epoch [1/100], Step [1100/13958], Loss: 2031414304309248.0000
Epoch [1/100], Step [1200/13958], Loss: 641778038341632.0000
Epoch [1/100], Step [1300/13958], Loss: 584688057974784.0000
Epoch [1/100], Step [1400/13958], Loss: 1328862747164672.0000
Epoch [1/100], Step [1500/13958], Loss: 1546635976900608.0000
Epoch [1/100], Step [1600/13958], Loss: 549643205214208.0000
Epoch [1/100], Step 

In [ ]:

import matplotlib.pyplot as plt


# Performance Tracking
training_losses = []
validation_losses = []

# Validation DataLoader: 'val_loader'
val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Reset optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Training Loop
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        train_loss = loss_fn(outputs, targets)

        if torch.isnan(train_loss):
            print("NaN loss encountered. Exiting training loop.")
            break

        # Backward pass and optimization
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Record training loss
        training_losses.append(train_loss.item())  

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {train_loss.item():.4f}')

    # Validation Loop
    with torch.no_grad():  # No gradient calculations during validation
        val_loss = 0.0
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss += loss_fn(outputs, targets).item() 
        val_loss /= len(val_loader)
        validation_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}')



Epoch [1/100], Step [100/13958], Loss: 4929602887348371720746214921076736.0000
Epoch [1/100], Step [200/13958], Loss: 4536415799710784950114404479795200.0000
Epoch [1/100], Step [300/13958], Loss: 4842755822862325510250804310704128.0000
Epoch [1/100], Step [400/13958], Loss: 4948193342403330097679443794329600.0000
Epoch [1/100], Step [500/13958], Loss: 6089998981542935417915795453247488.0000
Epoch [1/100], Step [600/13958], Loss: 6957215593143601432651428744134656.0000
Epoch [1/100], Step [700/13958], Loss: 4160193132886555209424742912098304.0000
Epoch [1/100], Step [800/13958], Loss: 5430571843476359767702255062482944.0000
Epoch [1/100], Step [900/13958], Loss: 5139074721395850917061895731019776.0000
Epoch [1/100], Step [1000/13958], Loss: 4718732181056500044719898100760576.0000
Epoch [1/100], Step [1100/13958], Loss: 5512288265469587719648346252509184.0000
Epoch [1/100], Step [1200/13958], Loss: 4783713201143638762074696755183616.0000
Epoch [1/100], Step [1300/13958], Loss: 335653607

In [ ]:


# Visualization
plt.plot(training_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()


In [ ]:

# Save the model
torch.save(model.state_dict(), 'model.pth')


In [ ]:

# evaluate the model
# model.eval()
# with torch.no_grad():
#     y_pred = model(X)
#     loss = loss_fn(y_pred, y.view(-1, 1))
#     print(f'Loss: {loss.item():.4f}')
    
#     # Print the first 5 predictions
#     y_pred = y_pred.cpu()
#     y = y.cpu()
#     print(y_pred[:5].numpy().flatten())
#     print(y[:5].numpy())